In [68]:
import os

In [69]:
%pwd

'C:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidneyDiseaseClassification'

In [70]:

os.chdir(r"C:\Users\quamr\OneDrive\Desktop\project\kidneyDiseaseClassification")

In [71]:
%pwd

'C:\\Users\\quamr\\OneDrive\\Desktop\\project\\kidneyDiseaseClassification'

In [72]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
   root_dir:Path
   source_url:str
   local_data_file:str
   unzip_dir:str

In [73]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [74]:
class ConfigurationManager:
  def __init__(
      self,
      config_filepath:Path=CONFIG_FILE_PATH,
      params_filepath:Path=PARAMS_FILE_PATH
  ):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)

    create_directories([self.config.artifacts_root])

  def get_data_ingestion_config(self) -> DataIngestionConfig:
    config = self.config.data_ingestion

    create_directories([config.root_dir])

    data_ingestion_config = DataIngestionConfig(
        root_dir = config.root_dir,
        source_url = config.source_url,
        local_data_file = config.local_data_file,
        unzip_dir = config.unzip_dir
    )
    return data_ingestion_config

In [75]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [76]:
class DataIngestion:
  def __init__(self, config: DataIngestionConfig):
    self.config = config

  def download_file(self,) -> str:
    """Fetch data from google drive
    """
    try:
      dataset_url = self.config.source_url
      zip_download_dir = self.config.local_data_file
      os.makedirs("artifacts/data_ingestion", exist_ok=True)
      logger.info(f"downloading data from {dataset_url} to {zip_download_dir}")

      file_id = dataset_url.split("/")[-2]
      prefix = "https://drive.google.com/uc?export=download&id="
      gdown.download(prefix+file_id, zip_download_dir)

      logger.info(f"Downloaded data from {dataset_url} to {zip_download_dir}")

    except Exception as e:
      raise e
  
  def extract_zip_file(self,)-> None:
    """Extract zip file to directory
    """
    unzip_path = self.config.unzip_dir
    os.makedirs(unzip_path, exist_ok=True)
    with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
      logger.info(f"Extracting zip file: {self.config.local_data_file} to dir: {unzip_path}")
      zip_ref.extractall(unzip_path)
      logger.info(f"Extracted zip file: {self.config.local_data_file} to dir: {unzip_path} and the size of data is {get_size(unzip_path)}")


In [77]:
try:
  config = ConfigurationManager()
  data_ingestion_config = config.get_data_ingestion_config()

  data_ingestion = DataIngestion(config=data_ingestion_config)
  data_ingestion.download_file()
  data_ingestion.extract_zip_file()
except Exception as e:
  raise e

[2025-12-01 00:25:21,876: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-01 00:25:21,877: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-01 00:25:21,878: INFO: common: created directory at: artifacts]
[2025-12-01 00:25:21,879: INFO: common: created directory at: artifacts/data_ingestion]
[2025-12-01 00:25:21,880: INFO: 4149466415: downloading data from https://drive.google.com/file/d/14ohz4rbdJ-0DGWf1TBtLy1Uvjp5PdLpa/view?usp=sharing to artifacts/data_ingestion/CT-kidney-data.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=14ohz4rbdJ-0DGWf1TBtLy1Uvjp5PdLpa
From (redirected): https://drive.google.com/uc?export=download&id=14ohz4rbdJ-0DGWf1TBtLy1Uvjp5PdLpa&confirm=t&uuid=30514988-7ff3-4b37-8d73-1a058bbc69e5
To: C:\Users\quamr\OneDrive\Desktop\project\kidneyDiseaseClassification\artifacts\data_ingestion\CT-kidney-data.zip
100%|██████████| 1.63G/1.63G [12:41<00:00, 2.13MB/s]

[2025-12-01 00:38:07,449: INFO: 4149466415: Downloaded data from https://drive.google.com/file/d/14ohz4rbdJ-0DGWf1TBtLy1Uvjp5PdLpa/view?usp=sharing to artifacts/data_ingestion/CT-kidney-data.zip]


[2025-12-01 00:38:07,574: INFO: 4149466415: Extracting zip file: artifacts/data_ingestion/CT-kidney-data.zip to dir: artifacts/data_ingestion]


EnsureError: Argument path of type <class 'str'> to <function get_size at 0x0000017AFF367A60> does not match annotation type <class 'pathlib.Path'>